In [ ]:
%pip install -q --upgrade ultralytics opencv-python numpy imutils tqdm


In [ ]:
# Self-contained run using panogeo.people_tracking
import sys, subprocess
from pathlib import Path
from datetime import datetime

# Import module API
from panogeo.people_tracking import run_tracking


VIDEO_PATH = "data/videos/onikuru_cropped_mini.mp4"

# Output directory
STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT_DIR = Path("output") / f"people_track_{STAMP}"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Model choices
MODEL_NAME = "yolo12m.pt"  # or a local Path

# Autodetect device
DEVICE = 0

# Tuned defaults (small-person friendly)
CONF_THRES = 0.08
IOU_THRES = 0.45
IMG_SIZE = 1920            # set to 3840 for full 4K, may need more VRAM
MAX_DET = 3000
PERSON_CLASS_ID = 0
AGNOSTIC_NMS = True
MAX_DISAPPEARED = 30
MAX_DISTANCE = 110.0
ENABLE_COUNTING = False
LINE_Y_FRACTION = 0.55
CENTER_CROP = (1920, 1080)
SHOW_TRAJ = True

print({
    "video": str(VIDEO_PATH),
    "out_dir": str(OUT_DIR),
    "device": DEVICE,
    "imgsz": IMG_SIZE,
    "conf": CONF_THRES,
    "iou": IOU_THRES,
})

# # Run
# module_out = run_tracking(
#     video_path=VIDEO_PATH,
#     output_path=OUT_DIR / "_module_full_traj_mini.mp4",
#     model=MODEL_NAME,
#     conf_thres=CONF_THRES,
#     iou_thres=IOU_THRES,
#     imgsz=IMG_SIZE,
#     max_det=MAX_DET,
#     person_class_id=PERSON_CLASS_ID,
#     agnostic_nms=AGNOSTIC_NMS,
#     device=DEVICE,
#     half=True,
#     amp=True,
#     enable_counting=ENABLE_COUNTING,
#     line_y_fraction=LINE_Y_FRACTION,
#     center_crop=CENTER_CROP,
#     show_trajectories=SHOW_TRAJ,
#     traj_max_points=400,
#     traj_thickness=2,
#     max_disappeared=MAX_DISAPPEARED,
#     max_distance=MAX_DISTANCE,
# )
# print(f"Saved via module: {module_out}")


{'video': 'data/videos/onikuru_cropped.mp4', 'out_dir': 'output\\people_track_20251209_114139', 'device': 0, 'imgsz': 1920, 'conf': 0.08, 'iou': 0.45}


In [ ]:
# Use case: Calibrate from a video (first frame) with the interactive UI, then solve calibration
from pathlib import Path

from panogeo import launch_calibration_ui
from panogeo.calibration import solve_calibration, save_calibration

# Video to calibrate from (first frame will be used)
try:
    VIDEO_FOR_CALIB = VIDEO_PATH  # reuse from above if defined
except NameError:
    VIDEO_FOR_CALIB = "data/videos/onikuru_cropped_mini.mp4"

# Where to save the clicked pixel↔geo pairs
CALIB_CSV = "output/calib_points_onikuru.csv"

# Projection and image shape
PROJECTION = "perspective"   # "pano" or "perspective"
IMG_W = 1920
IMG_H = 1080

# Map start center (approximate camera location) — adjust to your scene
CAM_LAT = 34.81613459114869
CAM_LON = 135.56935202298817
CAMERA_ALT_M = 20.0
GROUND_ALT_M = 0.0

# # Launch the interactive UI (click image then map; press "Save CSV")
# ui = launch_calibration_ui(
#     pano_path=str(VIDEO_FOR_CALIB),
#     map_center=(CAM_LAT, CAM_LON),
#     map_zoom=18,
#     display_width_px=IMG_W,     # render at full width for easier clicking
#     default_alt_m=GROUND_ALT_M, # default target altitude when clicking map
#     enable_zoom=True,
#     image_viewport_height_px=520,
#     projection=PROJECTION,
# )
# ui.display()


In [8]:
import os

# After saving CALIB_CSV in the UI, optionally run the solver below by setting RUN_SOLVE=True
RUN_SOLVE = True
OUT_DIR = Path("output")
DEM_XML_FOLDER = "data/dem/xml/FG-GML-523514-DEM1A-20250606"

# Choose calibration mode for perspective:
#   "monoplotting" - New: true ray-DEM intersection (59% more accurate than homography)
#   "homography"   - Legacy: 2D projective transform (faster, flat ground assumption)
PERSPECTIVE_MODE = "monoplotting"

if RUN_SOLVE:
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    if str(PROJECTION).strip().lower() == "perspective":
        out_npz = OUT_DIR / "calibration_perspective_onikuru.npz"
        
        if PERSPECTIVE_MODE == "monoplotting":
            # Monoplotting via ray-DEM intersection with ground error optimization
            # - Uses cv2.calibrateCamera for initial pose estimation
            # - Refines by directly minimizing ground error (not reprojection error)
            # - Achieves ~1.5m accuracy vs ~3.8m for homography (59% improvement)
            # - Image dimensions are auto-detected from the W/H columns in the calibration CSV
            from panogeo.perspective import (
                calibrate_monoplotting_with_dem,
                save_monoplotting_calibration,
            )
            
            calib, dem_grid = calibrate_monoplotting_with_dem(
                calib_csv=CALIB_CSV,
                # img_width/img_height auto-detected from CSV W/H columns
                dem_folder=(DEM_XML_FOLDER if DEM_XML_FOLDER else None),
                focal_length_px=None,  # Auto-estimate from image size
                use_ransac=True,
                ransac_reproj_thresh=8.0,
                optimize_ground_error=True,  # Refine by minimizing ground error (recommended)
            )
            
            if dem_grid is not None:
                N_axis, E_axis, ELEV = dem_grid
                save_monoplotting_calibration(str(out_npz), calib, N_axis, E_axis, ELEV)
                print(f"Saved monoplotting calibration to: {out_npz}")
                print(f"  Image size: {calib.img_width}x{calib.img_height}")
                print(f"  Camera position (ENU): {calib.cam_pos_enu}")
                print(f"  Reference: lat={calib.ref_lat:.8f}, lon={calib.ref_lon:.8f}")
                print(f"  DEM grid: {ELEV.shape[0]}x{ELEV.shape[1]}")
            else:
                save_monoplotting_calibration(str(out_npz), calib)
                print(f"Saved monoplotting calibration (no DEM) to: {out_npz}")
                print(f"  Image size: {calib.img_width}x{calib.img_height}")
        else:
            # LEGACY: Homography-based calibration (2D projective transform)
            from panogeo.perspective import solve_homography_from_csv, save_homography
            
            H, REF_LAT, REF_LON = solve_homography_from_csv(CALIB_CSV)
            save_homography(
                str(out_npz),
                H,
                ref_lat=REF_LAT,
                ref_lon=REF_LON,
                ground_alt_m=0.0,
                calib_csv=CALIB_CSV,
                google_api_key=os.getenv("GOOGLE_MAPS_API_KEY"),
                dem_xml_folder=(DEM_XML_FOLDER if DEM_XML_FOLDER else None),
                dem_spacing_m=1.0,   # set resolution by meters
                dem_margin_m=120.0,  # coverage beyond calib hull
            )
            print(f"Saved perspective homography to: {out_npz}")
    else:
        # Panoramas: solve for camera rotation and position
        from panogeo.calibration import solve_calibration, save_calibration
        res = solve_calibration(
            calib_csv=CALIB_CSV,
            cam_lat=CAM_LAT,
            cam_lon=CAM_LON,
            camera_alt_m=CAMERA_ALT_M,   # e.g., 20.0
            ground_alt_m=GROUND_ALT_M,
            default_width=IMG_W,
            default_height=IMG_H,
            optimize_cam_position=False,   # keep cam position fixed
            dem_path=(str(DEM_PATH) if 'DEM_PATH' in globals() and DEM_PATH else None),
            google_api_key=os.getenv("GOOGLE_MAPS_API_KEY"),
        )
        out_npz = OUT_DIR / "calibration_cam2enu_onikuru.npz"
        save_calibration(
            npz_path=str(out_npz),
            calib=res,
            cam_lat=res.cam_lat,
            cam_lon=res.cam_lon,
            camera_alt_m=res.camera_alt_m,
            ground_alt_m=GROUND_ALT_M,
        )
        print(f"Saved calibration to: {out_npz}")
        print(f"yaw={res.yaw_deg:.2f}°, pitch={res.pitch_deg:.2f}°, roll={res.roll_deg:.2f}°")

[monoplot] DEM elevations at control points: 10.5m to 14.3m
[monoplot] Ground error optimization: 1.47m -> 1.20m
[monoplot] Camera position (ENU): E=9.5, N=-110.8, U=32.0
[monoplot] Focal length: 3374.6px
Saved monoplotting calibration to: output\calibration_perspective_onikuru.npz
  Image size: 3840x2160
  Camera position (ENU): [     9.5374     -110.76      32.041]
  Reference: lat=34.81718319, lon=135.56929654
  DEM grid: 518x492


In [ ]:
# Geolocate tracked people and render a basemap video with trajectories (and PNG debug)
# 
# The geolocate_detections_perspective function auto-detects calibration type:
#   - "monoplotting": Ray-DEM intersection (~35k pts/sec, 1.5m accuracy, 59% better than homography)
#   - "homography": 2D projective transform (legacy, ~3.8m accuracy)
#
from pathlib import Path
from datetime import datetime
import os
import time
import pandas as pd

from panogeo.people_tracking import run_tracking
from panogeo.geolocate import geolocate_detections
from panogeo.perspective import geolocate_detections_perspective
from panogeo.mapplot import save_points_basemap, save_tracking_map_video
from panogeo.video import compose_side_by_side_video
import cv2

VIDEO_PATH = "data/videos/onikuru_cropped_mini.mp4"

# Output directory
STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT_DIR = Path("output") / f"people_track_{STAMP}"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Google Map Tiles API key (optional)
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
DEM_XML_FOLDER = os.getenv("DEM_XML_FOLDER")  # folder containing many GSI DEM *.xml tiles

# Model choices
MODEL_NAME = "yolo12m.pt"  # or a local Path

# Autodetect device
DEVICE = 0

# Tuned defaults (small-person friendly)
PROJECTION = "perspective"   # "pano" or "perspective"
CONF_THRES = 0.08
IOU_THRES = 0.45
IMG_SIZE = 1920            # set to 3840 for full 4K, may need more VRAM
MAX_DET = 3000
PERSON_CLASS_ID = 0
AGNOSTIC_NMS = True
MAX_DISAPPEARED = 30
MAX_DISTANCE = 110.0
ENABLE_COUNTING = False
LINE_Y_FRACTION = 0.55
CENTER_CROP = (1920, 1080)
SHOW_TRAJ = True
# Anti-switch gates for perspective mode
MAX_SPEED_PX_PER_FRAME = 32.0   # limit per-frame jump (px)
MIN_IOU_FOR_MATCH = 0.10        # require small overlap

print({
    "video": str(VIDEO_PATH),
    "out_dir": str(OUT_DIR),
    "device": DEVICE,
    "imgsz": IMG_SIZE,
    "conf": CONF_THRES,
    "iou": IOU_THRES,
})

# Paths
TRACK_EXPORT_CSV = OUT_DIR / "tracks_points_onikuru.csv"   # OUT_DIR from the tracking cell (timestamped)
CALIB_PANO_NPZ = Path("output") / "calibration_cam2enu_onikuru.npz"
CALIB_PERSP_NPZ = Path("output") / "calibration_perspective_onikuru.npz"

# 1) Re-run tracking with CSV export enabled (keeps the same video output path)
# Ensure we use the source video's FPS so the composed side-by-side stays in sync
cap0 = cv2.VideoCapture(str(VIDEO_PATH))
video_fps = cap0.get(cv2.CAP_PROP_FPS) or 30.0
cap0.close() if hasattr(cap0, "close") else cap0.release()
CAM_MP4 = OUT_DIR / "_module_full_traj_mini.mp4"

_ = run_tracking(
    video_path=VIDEO_PATH,
    output_path=CAM_MP4,
    model=MODEL_NAME,
    conf_thres=CONF_THRES,
    iou_thres=IOU_THRES,
    imgsz=IMG_SIZE,
    max_det=MAX_DET,
    person_class_id=PERSON_CLASS_ID,
    agnostic_nms=AGNOSTIC_NMS,
    device=DEVICE,
    half=True,
    amp=True,
    enable_counting=False,
    line_y_fraction=LINE_Y_FRACTION,
    center_crop=CENTER_CROP,
    show_trajectories=True,    # render trajectories for camera view
    traj_max_points=200,
    traj_thickness=5,
    max_disappeared=MAX_DISAPPEARED,
    max_distance=MAX_DISTANCE,
    max_speed_px_per_frame=MAX_SPEED_PX_PER_FRAME,
    min_iou_for_match=MIN_IOU_FOR_MATCH,
    export_csv_path=TRACK_EXPORT_CSV,
    show_progress=True,
    progress_desc="Tracking (center-crop)",
)

# Debug: check detection export W/H and a few samples
try:
    det_df = pd.read_csv(TRACK_EXPORT_CSV)
    w_set = sorted(det_df["W"].astype(int).unique().tolist())
    h_set = sorted(det_df["H"].astype(int).unique().tolist())
    print(f"[debug] tracks csv W unique={w_set}, H unique={h_set}, rows={len(det_df)}")
    print(det_df.head(3))
except Exception as e:
    print("[debug] failed reading tracks csv:", e)

# 2) Geolocate the exported track points (optimized vectorized processing)
try:
    t_start = time.perf_counter()
    
    if str(PROJECTION).strip().lower() == "perspective":
        xy_csv, geo_csv = geolocate_detections_perspective(
            detections_csv=str(TRACK_EXPORT_CSV),
            homography_npz=str(CALIB_PERSP_NPZ),
            output_dir=str(OUT_DIR),
            debug=True,
            calib_csv=(str(CALIB_CSV) if 'CALIB_CSV' in globals() else None),
            gate_margin_m=120.0,
            drop_outside=True,
            show_progress=True,
            progress_desc="Geolocate (perspective)",
            dem_xml_folder=(DEM_XML_FOLDER if DEM_XML_FOLDER else None),
            dem_margin_m=120.0,
            dem_spacing_m=5.0,
        )
    else:
        xy_csv, geo_csv = geolocate_detections(
            detections_csv=str(TRACK_EXPORT_CSV),
            calibration_npz=str(CALIB_PANO_NPZ),
            output_dir=str(OUT_DIR),
            dem_path=(str(DEM_PATH) if 'DEM_PATH' in globals() and DEM_PATH else None),
            show_progress=True,
            progress_desc="Geolocate (pano)",
            google_api_key=GOOGLE_MAPS_API_KEY,
            elev_rows=32,
            elev_cols=32,
            elev_margin_m=120.0,
        )
    
    t_elapsed = time.perf_counter() - t_start
    n_pts = len(pd.read_csv(TRACK_EXPORT_CSV))
    print(f"Geolocated CSV: {geo_csv}")
    print(f"Performance: {n_pts:,} points in {t_elapsed:.2f}s ({n_pts/t_elapsed:.0f} pts/sec)")
except Exception as e:
    print("[debug] geolocate failed:", e)
    raise

# Quick debug: show lon/lat and range stats
try:
    gdf = pd.read_csv(geo_csv)
    if not gdf.empty:
        print(
            "[debug] lon:[", float(gdf["lon"].min()), ",", float(gdf["lon"].max()), "]",
            "lat:[", float(gdf["lat"].min()), ",", float(gdf["lat"].max()), "]",
        )
        if "range_m" in gdf.columns:
            print(
                "[debug] range_m stats min/mean/max:",
                float(gdf["range_m"].min()),
                float(gdf["range_m"].mean()),
                float(gdf["range_m"].max()),
            )
        print(gdf.head(3))
except Exception as e:
    print("[debug] failed reading geo csv:", e)

# 3) Render a basemap video of geolocated tracks with trajectories
MAP_MP4 = OUT_DIR / "people_tracking_map_mini.mp4"
try:
    out_mp4 = save_tracking_map_video(
        geo_csv=str(geo_csv),
        out_mp4=str(MAP_MP4),
        provider="google",
        zoom=None,
        api_key=GOOGLE_MAPS_API_KEY,
        point_size=20.0,
        alpha=0.95,
        traj_max_frames=200,
        dpi=150,
        margin_frac=0.10,
        fps=video_fps,
        show_progress=True,
        progress_desc="Map video",
    )
    print("Saved map video:", out_mp4)
except Exception as e:
    print("[debug] map video failed:", e)

# Optional: also export single PNG basemap of all points
MAP_PNG = OUT_DIR / "people_tracking_map_carto.png"
out_png = save_points_basemap(
    geo_csv=str(geo_csv),
    out_png=str(MAP_PNG),
    provider="google",
    zoom=None,
    api_key=GOOGLE_MAPS_API_KEY,
    point_size=12.0,
    alpha=0.9,
    point_color="#FF5722",
    dpi=150,
)
print("Saved map:", out_png)

{'video': 'data/videos/onikuru_cropped.mp4', 'out_dir': 'output\\people_track_20251209_114205', 'device': 0, 'imgsz': 1920, 'conf': 0.08, 'iou': 0.45}
[debug] tracks csv W unique=[1920], H unique=[1080], rows=749013
                             video                    image  frame  track_id  \
0  data/videos/onikuru_cropped.mp4  onikuru_cropped_f000001      1         0   
1  data/videos/onikuru_cropped.mp4  onikuru_cropped_f000001      1         1   
2  data/videos/onikuru_cropped.mp4  onikuru_cropped_f000001      1         2   

      W     H  x_offset_px  y_offset_px  SRC_W  SRC_H    u_px    v_px  
0  1920  1080          960          540   3840   2160  1122.0   396.0  
1  1920  1080          960          540   3840   2160  1807.0  1021.0  
2  1920  1080          960          540   3840   2160  1835.0   794.0  
[persp] calibration type: monoplotting
[persp] applied crop offsets: mean x=960.0, y=540.0
[persp] monoplotting: cam_pos=[     9.5374     -110.76      32.041]
[persp] ref: lat

In [ ]:
# 5) Compose side-by-side video: camera view (with trajectories) + map view (with trajectories)
from panogeo.video import compose_side_by_side_video
import cv2
from pathlib import Path

# OUT_DIR = Path("C:/Users/kunih/OneDrive/00_Codes/python/panogeo/output/people_track_20251205_140947")

cam_video = str(OUT_DIR / "_module_full_traj_mini.mp4")
map_video = str(OUT_DIR / "people_tracking_map_mini.mp4")
# Write MP4; function will try H.264/avc1 first and fall back if needed
out_video = str(OUT_DIR / "people_tracking_split_mini.mp4")

# Use camera video's fps for consistent playback
cap = cv2.VideoCapture(cam_video)
compose_fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
cap.release()

out_path = compose_side_by_side_video(cam_video, map_video, out_video, layout="h", fps=compose_fps, gap=8)
print("Saved combined video:", out_path)

# Fallback (uncomment if MP4 is not playable in your default player)
# out_video_avi = str(OUT_DIR / "people_tracking_split.avi")
# out_path = compose_side_by_side_video(cam_video, map_video, out_video_avi, layout="h", fps=compose_fps, gap=8, codec="XVID")
# print("Saved combined video (AVI):", out_path)


[compose] using codec=avc1, size=(2536x1080) @ 30.00fps
[compose] frame 30/5417
[compose] frame 60/5417
[compose] frame 90/5417
[compose] frame 120/5417
[compose] frame 150/5417
[compose] frame 180/5417
[compose] frame 210/5417
[compose] frame 240/5417
[compose] frame 270/5417
[compose] frame 300/5417
[compose] frame 330/5417
[compose] frame 360/5417
[compose] frame 390/5417
[compose] frame 420/5417
[compose] frame 450/5417
[compose] frame 480/5417
[compose] frame 510/5417
[compose] frame 540/5417
[compose] frame 570/5417
[compose] frame 600/5417
[compose] frame 630/5417
[compose] frame 660/5417
[compose] frame 690/5417
[compose] frame 720/5417
[compose] frame 750/5417
[compose] frame 780/5417
[compose] frame 810/5417
[compose] frame 840/5417
[compose] frame 870/5417
[compose] frame 900/5417
[compose] frame 930/5417
[compose] frame 960/5417
[compose] frame 990/5417
[compose] frame 1020/5417
[compose] frame 1050/5417
[compose] frame 1080/5417
[compose] frame 1110/5417
[compose] frame 11

In [11]:
## verify downloader
